## Loading Packages

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml import Pipeline
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit, expr, avg
from pyspark.sql.functions import avg
from pyspark.sql.types import FloatType, IntegerType


## Reading data set 
The data is read to a data frame with the header filed set to true, where the headers will also be read and the infer schema option is enabled where the data frame automatically infers the schema of different columns

In [4]:
zomato_orgnl=spark.read.csv("/FileStore/tables/cleanedzomatodata.csv",header = 'True',inferSchema='True')

## Printing Schema 
Below displayed is the schema of different columns inferred by the data frame

In [6]:
zomato_orgnl.printSchema()

root
-- name: string (nullable = true)
-- online_order: string (nullable = true)
-- book_table: string (nullable = true)
-- rate: string (nullable = true)
-- votes: integer (nullable = true)
-- rest_type: string (nullable = true)
-- dish_liked: string (nullable = true)
-- cuisines: string (nullable = true)
-- average_cost: integer (nullable = true)
-- restaurant_type: string (nullable = true)
-- locality: string (nullable = true)

## Data Cleaning

### Counting null values
The null values present in different columns of the entire data is calculated using a function. Below provided is a function that counts the null values present according to columns

In [9]:
def null_value_count(df):
  null_columns_counts = []
  numRows = df.count()
  for k in df.columns:
    nullRows = df.where(col(k).isNull()).count()
    if(nullRows > 0):
      temp = k,nullRows
      null_columns_counts.append(temp)
  return(null_columns_counts)

The function is called, where the parameter passed is the data frame. This allows the function to calculate the number of null values in each column of the data frame

In [11]:
null_columns_count_list = null_value_count(zomato_orgnl)


The count of null values are displayed according to columns

In [13]:
spark.createDataFrame(null_columns_count_list, ['Column_With_Null_Value', 'Null_Values_Count']).show()

+----------------------+-----------------+
Column_With_Null_Value|Null_Values_Count|
+----------------------+-----------------+
 rate| 7775|
 rest_type| 227|
 dish_liked| 28078|
 cuisines| 45|
 average_cost| 346|
+----------------------+-----------------+

As the column dish_liked have huge number counts of null values, it is dropped to avoid further complexities in analysis

In [15]:
zomato_orgnl = zomato_orgnl.drop("dish_liked")

Printing the schema to see the rest available columns in the data frame

In [17]:
zomato_orgnl.printSchema()

root
-- name: string (nullable = true)
-- online_order: string (nullable = true)
-- book_table: string (nullable = true)
-- rate: string (nullable = true)
-- votes: integer (nullable = true)
-- rest_type: string (nullable = true)
-- cuisines: string (nullable = true)
-- average_cost: integer (nullable = true)
-- restaurant_type: string (nullable = true)
-- locality: string (nullable = true)

The unique values of rate are displayed as column rate is the label and it is necessary to check whether there are any irrelevant values in that column

In [19]:
rate_new=zomato_orgnl.select("rate").dropDuplicates()
rate_new.show(70, False)

+----+
rate|
+----+
2.6 |
3.1 |
4.2 |
3 |
3.8 |
4.4 |
2.7 |
2.9 |
4.5 |
null|
2.5 |
2.4 |
4.9 |
3.4 |
3.3 |
1.8 |
4.3 |
3.5 |
4.8 |
4.1 |
4.6 |
NEW |
- |
3.6 |
2.8 |
3.2 |
2.2 |
3.7 |
4.7 |
4 |
2.3 |
2.1 |
2 |
3.9 |
+----+

### Filling Missisng Fields with Mean
As there are many null vaues in the columns rate and average cost, the mean values of both the columns are calculated to fill the empty filed in the column with the mean value of that column

In [21]:
mean_rate=zomato_orgnl.select([mean('rate')])
mean_rate.show()
mean_average_cost=zomato_orgnl.select([mean('average_cost')])
mean_average_cost.show()
zomato_orgnl = zomato_orgnl.withColumn("rate",when((zomato_orgnl["rate"].isNull()), 3.7).otherwise(zomato_orgnl["rate"]))
zomato_orgnl = zomato_orgnl.withColumn("average_cost",when((zomato_orgnl["average_cost"].isNull()), 561).otherwise(zomato_orgnl["average_cost"]))
zomato_orgnl.show(120)

+------------------+
 avg(rate)|
+------------------+
3.7004488179527266|
+------------------+

+-----------------+
avg(average_cost)|
+-----------------+
555.4315664479959|
+-----------------+

+--------------------+------------+----------+----+-----+--------------------+--------------------+------------+---------------+------------+
 name|online_order|book_table|rate|votes| rest_type| cuisines|average_cost|restaurant_type| locality|
+--------------------+------------+----------+----+-----+--------------------+--------------------+------------+---------------+------------+
 Jalsa| Yes| Yes| 4.1| 775| Casual Dining|North Indian, Mug...| 800| Buffet|Banashankari|
 Spice Elephant| Yes| No| 4.1| 787| Casual Dining|Chinese, North In...| 800| Buffet|Banashankari|
 San Churro Cafe| Yes| No| 3.8| 918| Cafe, Casual Dining|Cafe, Mexican, It...| 800| Buffet|Banashankari|
Addhuri Udupi Bho...| No| No| 3.7| 88| Quick Bites|South Indian, Nor...| 300| Buffet|Banashankari|
 Grand Village| No| No| 3.8| 166| Casual Dining|North Indian, Raj...| 600| Buffet|Banashankari|
 Timepass Dinner| Yes| No| 3.8| 286| Casual Dining| North Indian| 600| Buffet|Banashankari|
Rosewood Internat...| No| No| 3.6| 8| Casual Dining|North Indian, Sou...| 800| Buffet|Banashankari|
 Onesta| Yes| Yes| 4.6| 2556| Casual Dining, Cafe|Pizza, Cafe, Italian| 600| Cafes|Banashankari|
 Penthouse Cafe| Yes| No| 4| 324| Cafe|Cafe, Italian, Co...| 700| Cafes|Banashankari|
 Smacznego| Yes| No| 4.2| 504| Cafe|Cafe, Mexican, It...| 550| Cafes|Banashankari|
CafÃƒÂƒÃ‚ÂƒÃƒÂ‚Ã‚...| Yes| No| 4.1| 402| Cafe| Cafe| 500| Cafes|Banashankari|
 Cafe Shuffle| Yes| Yes| 4.2| 150| Cafe|Cafe, Italian, Co...| 600| Cafes|Banashankari|
 The Coffee Shack| Yes| Yes| 4.2| 164| Cafe|Cafe, Chinese, Co...| 500| Cafes|Banashankari|
 Caf-Eleven| No| No| 4| 424| Cafe| Cafe, Continental| 450| Cafes|Banashankari|
 San Churro Cafe| Yes| No| 3.8| 918| Cafe, Casual Dining|Cafe, Mexican, It...| 800| Cafes|Banashankari|
 Cafe Vivacity| Yes| No| 3.8| 90| Cafe| Cafe| 650| Cafes|Banashankari|
 Catch-up-ino| Yes| No| 3.9| 133| Cafe|Cafe, Fast Food, ...| 800| Cafes|Banashankari|
 Kirthi's Biryani| Yes| No| 3.8| 144| Cafe|Chinese, Cafe, It...| 700| Cafes|Banashankari|
 T3H Cafe| No| No| 3.9| 93| Cafe|Cafe, Italian, Am...| 300| Cafes|Banashankari|
360 Atoms Restaur...| Yes| No| 3.1| 13| Cafe|Cafe, Chinese, Co...| 400| Cafes|Banashankari|
 The Vintage Cafe| Yes| No| 3| 62| Cafe|Cafe, French, Nor...| 400| Cafes|Banashankari|
 Woodee Pizza| Yes| No| 3.7| 180| Cafe|Cafe, Pizza, Fast...| 500| Cafes|Banashankari|
 Cafe Coffee Day| No| No| 3.6| 28| Cafe| Cafe, Fast Food| 900| Cafes|Banashankari|
 My Tea House| Yes| No| 3.6| 62| Quick Bites, Cafe|Italian, Fast Foo...| 600| Cafes|Banashankari|
 Hide Out Cafe| No| No| 3.7| 31| Cafe| Cafe| 300| Cafes|Banashankari|
 CAFE NOVA| No| No| 3.2| 11| Cafe| Cafe, Bakery| 600| Cafes|Banashankari|
 Coffee Tindi| Yes| No| 3.8| 75| Cafe, Quick Bites| Cafe, South Indian| 200| Cafes|Banashankari|
 Sea Green Cafe| No| No| 3.3| 4| Cafe|Cafe, Fast Food, ...| 500| Cafes|Banashankari|
 Cuppa| No| No| 3.3| 23| Cafe| Cafe, Fast Food| 550| Cafes|Banashankari|
 Srinathji's Cafe| No| No| 3.8| 148| Casual Dining, Cafe|North Indian, Caf...| 550| Cafes|Banashankari|
 Redberrys| Yes| No| 4| 219| Cafe| Cafe, Italian| 600| Cafes|Banashankari|
 Foodiction| Yes| No| 2.8| 506| Quick Bites|North Indian, Fas...| 500| Delivery|Banashankari|
 Sweet Truth| Yes| No| 3.9| 35| Delivery| Bakery, Desserts| 500| Delivery|Banashankari|
 Ovenstory Pizza| Yes| No| 3.9| 172| Delivery| Pizza| 750| Delivery|Banashankari|
 Faasos| Yes| No| 4.2| 415| Quick Bites|North Indian, Bir...| 500| Delivery|Banashankari|
 Behrouz Biryani| Yes| No| 3.9| 230| Delivery| Biryani| 650| Delivery|Banashankari|
 Fast And Fresh| Yes| No| 2.8| 91| Quick Bites|North Indian, Chi...| 400| Delivery|Banashankari|
 Szechuan Dragon| Yes| No| 4.2| 1647| Casual Dining|Chinese, Thai, Momos| 600| Delivery|Banashankari|
 Empire Restaurant| Yes| No| 4.4| 4884| Casual

### Removing Irrelevant values
As there are values like `NEW` and `-` in the rate column, where the column should only have float values, those values are removed from the column

In [23]:
zomato_orgnl = zomato_orgnl[zomato_orgnl.rate != 'NEW']
zomato_orgnl = zomato_orgnl[zomato_orgnl.rate != '-']


After removing the irrelevant values and filling the missing fields with mean value, the unique values of column `rate` are displayed again to see whether there are any complexities

In [25]:
rate_new=zomato_orgnl.select("rate").dropDuplicates()
rate_new.show(70, False)

+----+
rate|
+----+
2.6 |
3.1 |
4.2 |
3 |
3.8 |
4.4 |
2.7 |
2.9 |
4.5 |
2.5 |
2.4 |
4.9 |
3.4 |
3.3 |
1.8 |
4.3 |
3.5 |
4.8 |
4.1 |
4.6 |
3.6 |
2.8 |
3.2 |
2.2 |
3.7 |
4.7 |
4 |
2.3 |
2.1 |
2 |
3.9 |
+----+

### Type Conversion
The type of columns `rate` and `average cost` are inferred as strings, which are changed to type float

In [27]:
zomato_orgnl=zomato_orgnl.withColumn("rate", zomato_orgnl['rate'].cast(FloatType()))
zomato_orgnl=zomato_orgnl.withColumn("average_cost", zomato_orgnl['average_cost'].cast(FloatType()))
zomato_orgnl.printSchema()

root
-- name: string (nullable = true)
-- online_order: string (nullable = true)
-- book_table: string (nullable = true)
-- rate: float (nullable = true)
-- votes: integer (nullable = true)
-- rest_type: string (nullable = true)
-- cuisines: string (nullable = true)
-- average_cost: float (nullable = true)
-- restaurant_type: string (nullable = true)
-- locality: string (nullable = true)

## Exploratory Data Analysis

The relation between location, average_cost and rating is visualized to check whether the rating and the avregae_cost of restaurants increases with respect to the area where the restaurant is located

In [30]:
display(zomato_orgnl)

name,online_order,book_table,rate,votes,rest_type,cuisines,average_cost,restaurant_type,locality
Jalsa,Yes,Yes,4.1,775,Casual Dining,"North Indian, Mughlai, Chinese",800.0,Buffet,Banashankari
Spice Elephant,Yes,No,4.1,787,Casual Dining,"Chinese, North Indian, Thai",800.0,Buffet,Banashankari
San Churro Cafe,Yes,No,3.8,918,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,Buffet,Banashankari
Addhuri Udupi Bhojana,No,No,3.7,88,Quick Bites,"South Indian, North Indian",300.0,Buffet,Banashankari
Grand Village,No,No,3.8,166,Casual Dining,"North Indian, Rajasthani",600.0,Buffet,Banashankari
Timepass Dinner,Yes,No,3.8,286,Casual Dining,North Indian,600.0,Buffet,Banashankari
Rosewood International Hotel - Bar & Restaurant,No,No,3.6,8,Casual Dining,"North Indian, South Indian, Andhra, Chinese",800.0,Buffet,Banashankari
Onesta,Yes,Yes,4.6,2556,"Casual Dining, Cafe","Pizza, Cafe, Italian",600.0,Cafes,Banashankari
Penthouse Cafe,Yes,No,4.0,324,Cafe,"Cafe, Italian, Continental",700.0,Cafes,Banashankari
Smacznego,Yes,No,4.2,504,Cafe,"Cafe, Mexican, Italian, Momos, Beverages",550.0,Cafes,Banashankari


A visualization is created to check whether the rating of the restaurants increase when the restaurant provides an online ordering option

In [32]:
display(zomato_orgnl)

name,online_order,book_table,rate,votes,rest_type,cuisines,average_cost,restaurant_type,locality
Jalsa,Yes,Yes,4.1,775,Casual Dining,"North Indian, Mughlai, Chinese",800.0,Buffet,Banashankari
Spice Elephant,Yes,No,4.1,787,Casual Dining,"Chinese, North Indian, Thai",800.0,Buffet,Banashankari
San Churro Cafe,Yes,No,3.8,918,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,Buffet,Banashankari
Addhuri Udupi Bhojana,No,No,3.7,88,Quick Bites,"South Indian, North Indian",300.0,Buffet,Banashankari
Grand Village,No,No,3.8,166,Casual Dining,"North Indian, Rajasthani",600.0,Buffet,Banashankari
Timepass Dinner,Yes,No,3.8,286,Casual Dining,North Indian,600.0,Buffet,Banashankari
Rosewood International Hotel - Bar & Restaurant,No,No,3.6,8,Casual Dining,"North Indian, South Indian, Andhra, Chinese",800.0,Buffet,Banashankari
Onesta,Yes,Yes,4.6,2556,"Casual Dining, Cafe","Pizza, Cafe, Italian",600.0,Cafes,Banashankari
Penthouse Cafe,Yes,No,4.0,324,Cafe,"Cafe, Italian, Continental",700.0,Cafes,Banashankari
Smacznego,Yes,No,4.2,504,Cafe,"Cafe, Mexican, Italian, Momos, Beverages",550.0,Cafes,Banashankari


The relationship between the type of restaurant and the location is evaluated as the people living in some locality may only prefer restaurants of certain types

In [34]:
display(zomato_orgnl)

name,online_order,book_table,rate,votes,rest_type,cuisines,average_cost,restaurant_type,locality
Jalsa,Yes,Yes,4.1,775,Casual Dining,"North Indian, Mughlai, Chinese",800.0,Buffet,Banashankari
Spice Elephant,Yes,No,4.1,787,Casual Dining,"Chinese, North Indian, Thai",800.0,Buffet,Banashankari
San Churro Cafe,Yes,No,3.8,918,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,Buffet,Banashankari
Addhuri Udupi Bhojana,No,No,3.7,88,Quick Bites,"South Indian, North Indian",300.0,Buffet,Banashankari
Grand Village,No,No,3.8,166,Casual Dining,"North Indian, Rajasthani",600.0,Buffet,Banashankari
Timepass Dinner,Yes,No,3.8,286,Casual Dining,North Indian,600.0,Buffet,Banashankari
Rosewood International Hotel - Bar & Restaurant,No,No,3.6,8,Casual Dining,"North Indian, South Indian, Andhra, Chinese",800.0,Buffet,Banashankari
Onesta,Yes,Yes,4.6,2556,"Casual Dining, Cafe","Pizza, Cafe, Italian",600.0,Cafes,Banashankari
Penthouse Cafe,Yes,No,4.0,324,Cafe,"Cafe, Italian, Continental",700.0,Cafes,Banashankari
Smacznego,Yes,No,4.2,504,Cafe,"Cafe, Mexican, Italian, Momos, Beverages",550.0,Cafes,Banashankari


The relationship between `cuisines` and `location` is analyzed to check whether the people living in some localities only prefer a specific cuisine

In [36]:
display(zomato_orgnl)

name,online_order,book_table,rate,votes,rest_type,cuisines,average_cost,restaurant_type,locality
Jalsa,Yes,Yes,4.1,775,Casual Dining,"North Indian, Mughlai, Chinese",800.0,Buffet,Banashankari
Spice Elephant,Yes,No,4.1,787,Casual Dining,"Chinese, North Indian, Thai",800.0,Buffet,Banashankari
San Churro Cafe,Yes,No,3.8,918,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,Buffet,Banashankari
Addhuri Udupi Bhojana,No,No,3.7,88,Quick Bites,"South Indian, North Indian",300.0,Buffet,Banashankari
Grand Village,No,No,3.8,166,Casual Dining,"North Indian, Rajasthani",600.0,Buffet,Banashankari
Timepass Dinner,Yes,No,3.8,286,Casual Dining,North Indian,600.0,Buffet,Banashankari
Rosewood International Hotel - Bar & Restaurant,No,No,3.6,8,Casual Dining,"North Indian, South Indian, Andhra, Chinese",800.0,Buffet,Banashankari
Onesta,Yes,Yes,4.6,2556,"Casual Dining, Cafe","Pizza, Cafe, Italian",600.0,Cafes,Banashankari
Penthouse Cafe,Yes,No,4.0,324,Cafe,"Cafe, Italian, Continental",700.0,Cafes,Banashankari
Smacznego,Yes,No,4.2,504,Cafe,"Cafe, Mexican, Italian, Momos, Beverages",550.0,Cafes,Banashankari


The count of Restaurants according to the locality is provided below, which represents whether locality is an important factor for restaurants

In [38]:
numberofrestaurants = zomato_orgnl.groupBy("locality").count()
display(numberofrestaurants)

locality,count
Bellandur,1153
Indiranagar,1741
BTM,3108
Banashankari,823
Koramangala 7th Block,2824
JP Nagar,1965
Lavelle Road,1702
Kammanahalli,1282
Jayanagar,2292
Brigade Road,1715


### Feature Transformation
 As there are many categorical values in the data, which also cats as features for predicting the restaurant rating, the categorical columns are label indexed using `StringIndexer`and then encoded to binary vectors using `OneHotEncoderEstimator`. As all the features required for prediction need to be merged, `VectorAssembler` is used to merge all vectors along with the columns containing numerical values.

In [40]:
cols = zomato_orgnl.columns
categoricalColumns = ['online_order', 'book_table', 'rest_type', 'cuisines', 'locality']
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index', handleInvalid = "keep")
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
label_stringIdx = StringIndexer(inputCol = 'rate', outputCol = 'label')
stages += [label_stringIdx]
numericCols = ['votes', 'average_cost']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]    

Pipelines are used to perform feature transformation on the data frame

In [42]:
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(zomato_orgnl)
zomato_orgnl = pipelineModel.transform(zomato_orgnl)
selectedCols = ['label', 'features'] + cols
zomato_orgnl = zomato_orgnl.select(selectedCols)


There is now a new `features` and `label` column appeared on the data frame after performing the feature transformation

In [44]:
zomato_orgnl.printSchema()

root
-- label: double (nullable = false)
-- features: vector (nullable = true)
-- name: string (nullable = true)
-- online_order: string (nullable = true)
-- book_table: string (nullable = true)
-- rate: float (nullable = true)
-- votes: integer (nullable = true)
-- rest_type: string (nullable = true)
-- cuisines: string (nullable = true)
-- average_cost: float (nullable = true)
-- restaurant_type: string (nullable = true)
-- locality: string (nullable = true)

The entire data is now split to training and testing data where 70% of the entire data is used for training purpose and the rest 30% is used for testing

In [46]:
train, test = zomato_orgnl.randomSplit([0.7, 0.3], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 34588
Test Dataset Count: 14852

##Models and Implementation

### Logistic Regression
Applying Logistic Regression to train the model using the `train` data set and the trained model is applied for testing on the `test` data set

In [49]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(train)
predictions = lrModel.transform(test)
predictions.select('average_cost', 'votes', 'label', 'rawPrediction', 'prediction', 'probability').show(10)

+------------+-----+-----+--------------------+----------+--------------------+
average_cost|votes|label| rawPrediction|prediction| probability|
+------------+-----+-----+--------------------+----------+--------------------+
 150.0| 95| 0.0|[3.70498466611429...| 0.0|[0.25757745715873...|
 150.0| 117| 0.0|[3.68464510737936...| 0.0|[0.25451073110509...|
 200.0| 34| 0.0|[3.65185106300535...| 0.0|[0.24679378509965...|
 200.0| 0| 0.0|[3.68328492650479...| 0.0|[0.25137468219996...|
 250.0| 0| 0.0|[3.64811769873377...| 0.0|[0.24659057650406...|
 400.0| 0| 0.0|[3.54261601542069...| 0.0|[0.23225202419985...|
 150.0| 95| 0.0|[3.68951547372246...| 0.0|[0.25467416872768...|
 350.0| 0| 0.0|[3.63667647535738...| 0.0|[0.24815998412895...|
 400.0| 0| 0.0|[3.60150924758635...| 0.0|[0.24316684448966...|
 350.0| 48| 0.0|[3.43969136772584...| 0.0|[0.21520653313388...|
+------------+-----+-----+--------------------+----------+--------------------+
only showing top 10 rows

###Accuracy Checking 
The accuracy of the predicted value is checked by determining the `Area Under ROC`Curve

In [51]:
evaluator = BinaryClassificationEvaluator()
print('Test Area Under ROC', evaluator.evaluate(predictions))

Test Area Under ROC 0.5910089596691193

### Decision Tree
Applying Decision Tree to train the model using the `train` data set and the trained model is applied for testing on the `test` data set

In [53]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
dtModel = dt.fit(train)
predictions = dtModel.transform(test)
predictions.select('average_cost', 'votes', 'label', 'rawPrediction', 'prediction', 'probability').show()


+------------+-----+-----+--------------------+----------+--------------------+
average_cost|votes|label| rawPrediction|prediction| probability|
+------------+-----+-----+--------------------+----------+--------------------+
 150.0| 95| 0.0|[1975.0,1323.0,18...| 0.0|[0.16572962994042...|
 150.0| 117| 0.0|[1975.0,1323.0,18...| 0.0|[0.16572962994042...|
 200.0| 34| 0.0|[1975.0,1323.0,18...| 0.0|[0.16572962994042...|
 200.0| 0| 0.0|[5397.0,0.0,1.0,0...| 0.0|[0.99907441688263...|
 250.0| 0| 0.0|[5397.0,0.0,1.0,0...| 0.0|[0.99907441688263...|
 400.0| 0| 0.0|[5397.0,0.0,1.0,0...| 0.0|[0.99907441688263...|
 150.0| 95| 0.0|[1975.0,1323.0,18...| 0.0|[0.16572962994042...|
 350.0| 0| 0.0|[5397.0,0.0,1.0,0...| 0.0|[0.99907441688263...|
 400.0| 0| 0.0|[5397.0,0.0,1.0,0...| 0.0|[0.99907441688263...|
 350.0| 48| 0.0|[1975.0,1323.0,18...| 0.0|[0.16572962994042...|
 200.0| 69| 0.0|[1975.0,1323.0,18...| 0.0|[0.16572962994042...|
 350.0| 0| 0.0|[5397.0,0.0,1.0,0...| 0.0|[0.99907441688263...|
 300.0| 40| 0.0|[1975.0,1323.0,18...| 0.0|[0.16572962994042...|
 150.0| 117| 0.0|[1975.0,1323.0,18...| 0.0|[0.16572962994042...|
 400.0| 195| 0.0|[425.0,1385.0,831...| 5.0|[0.03903021397740...|
 400.0| 195| 0.0|[425.0,1385.0,831...| 5.0|[0.03903021397740...|
 250.0| 0| 0.0|[5397.0,0.0,1.0,0...| 0.0|[0.99907441688263...|
 350.0| 0| 0.0|[5397.0,0.0,1.0,0...| 0.0|[0.99907441688263...|
 800.0| 19| 0.0|[1975.0,1323.0,18...| 0.0|[0.16572962994042...|
 400.0| 195| 0.0|[425.0,1385.0,831...| 5.0|[0.03903021397740...|
+------------+-----+-----+--------------------+----------+--------------------+
only showing top 20 rows

###Accuracy Checking 
The accuracy of the predicted value is checked by determining the `Area Under ROC`Curve

In [55]:
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

Test Area Under ROC: 0.8153630883107873

### Random Forest
Applying Random Forest to train the model using the `train` data set and the trained model is applied for testing on the `test` data set

In [57]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')
rfModel = rf.fit(train)
predictions = rfModel.transform(test)
predictions.select('average_cost', 'votes', 'label', 'rawPrediction', 'prediction', 'probability').show(10)



+------------+-----+-----+--------------------+----------+--------------------+
average_cost|votes|label| rawPrediction|prediction| probability|
+------------+-----+-----+--------------------+----------+--------------------+
 150.0| 95| 0.0|[4.97568128634912...| 0.0|[0.24878406431745...|
 150.0| 117| 0.0|[4.97568128634912...| 0.0|[0.24878406431745...|
 200.0| 34| 0.0|[4.97568128634912...| 0.0|[0.24878406431745...|
 200.0| 0| 0.0|[4.97568128634912...| 0.0|[0.24878406431745...|
 250.0| 0| 0.0|[4.97568128634912...| 0.0|[0.24878406431745...|
 400.0| 0| 0.0|[4.97568128634912...| 0.0|[0.24878406431745...|
 150.0| 95| 0.0|[4.97568128634912...| 0.0|[0.24878406431745...|
 350.0| 0| 0.0|[4.97568128634912...| 0.0|[0.24878406431745...|
 400.0| 0| 0.0|[4.97568128634912...| 0.0|[0.24878406431745...|
 350.0| 48| 0.0|[4.97568128634912...| 0.0|[0.24878406431745...|
+------------+-----+-----+--------------------+----------+--------------------+
only showing top 10 rows

###Accuracy Checking
The accuracy of the predicted value is checked by determining the `Area Under ROC`Curve

In [59]:
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

Test Area Under ROC: 0.6755631722731427

It is evident from the above analysis that, Decision Tree performed better prediction when compared to Logistic Regression and Random Forest

## Data Visualization

The data is visualized to find the count of restaurants that belongs to certain types

In [63]:
restauranttypes = zomato_orgnl.groupBy("rest_type").count()
display(restauranttypes)

rest_type,count
"Quick Bites, Mess",4
"Casual Dining, Sweet Shop",8
Fine Dining,346
"Beverage Shop, Dessert Parlor",79
"Casual Dining, Pub",127
"Dessert Parlor, Bakery",85
Dhaba,33
"Fine Dining, Bar",40
"Lounge, Bar",36
Bakery,1070


The data is visualized to see the number of restaurants that provide the facility to book a table in advance. It is evident from the visualization that, most restaurants of Banglore does not provide the facility of booking tables.

In [65]:
booktablefacility = zomato_orgnl.groupBy("book_table").count()
display(booktablefacility)

book_table,count
No,43120
Yes,6320


The data is displayed to show that the rating of the restaurant is dependant on online ordering option. It is evident from the below analysis that the restaurants that have online ordering options available hold a high rating when compared to the restaurants that does not have that facility.

In [67]:
group_data = zomato_orgnl.groupBy("online_order")
group_data.agg({'rate':'min'}).show()


+------------+---------+
online_order|min(rate)|
+------------+---------+
 No| 1.8|
 Yes| 2.1|
+------------+---------+

The data is grouped based on average cost where the maximum ratings obtained by restaurants with different amount was determined

In [69]:
averagecost = zomato_orgnl.groupBy('average_cost')
averagecost.agg({'rate':'max'}).show()

+------------+---------+
average_cost|max(rate)|
+------------+---------+
 550.0| 4.7|
 500.0| 4.6|
 360.0| 3.7|
 190.0| 3.7|
 180.0| 4.3|
 3200.0| 4.4|
 1100.0| 4.6|
 350.0| 4.7|
 100.0| 4.8|
 1500.0| 4.9|
 6000.0| 4.3|
 1700.0| 4.7|
 2700.0| 4.5|
 1250.0| 4.5|
 1800.0| 4.8|
 1050.0| 2.6|
 70.0| 3.3|
 1400.0| 4.9|
 240.0| 3.3|
 2100.0| 4.8|
+------------+---------+
only showing top 20 rows

##Finding the Best Restaurant

### Creating Temporary Table
Inorder to determine the best restaurants, various querying is performed on the data set for which the data needs to be displayed as an sql table where querying can be easily performed

In [72]:
temp_table_name="zomato"
zomato_orgnl.createOrReplaceTempView(temp_table_name)

Displaying the temporary table created

In [74]:
%sql
SELECT * from zomato

label,features,name,online_order,book_table,rate,votes,rest_type,cuisines,average_cost,restaurant_type,locality
5.0,"List(0, 2761, List(0, 3, 5, 162, 2757, 2759, 2760), List(1.0, 1.0, 1.0, 1.0, 1.0, 775.0, 800.0))",Jalsa,Yes,Yes,4.1,775,Casual Dining,"North Indian, Mughlai, Chinese",800.0,Buffet,Banashankari
5.0,"List(0, 2761, List(0, 2, 5, 1144, 2757, 2759, 2760), List(1.0, 1.0, 1.0, 1.0, 1.0, 787.0, 800.0))",Spice Elephant,Yes,No,4.1,787,Casual Dining,"Chinese, North Indian, Thai",800.0,Buffet,Banashankari
2.0,"List(0, 2761, List(0, 2, 27, 810, 2757, 2759, 2760), List(1.0, 1.0, 1.0, 1.0, 1.0, 918.0, 800.0))",San Churro Cafe,Yes,No,3.8,918,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,Buffet,Banashankari
0.0,"List(0, 2761, List(1, 2, 4, 113, 2757, 2759, 2760), List(1.0, 1.0, 1.0, 1.0, 1.0, 88.0, 300.0))",Addhuri Udupi Bhojana,No,No,3.7,88,Quick Bites,"South Indian, North Indian",300.0,Buffet,Banashankari
2.0,"List(0, 2761, List(1, 2, 5, 375, 2757, 2759, 2760), List(1.0, 1.0, 1.0, 1.0, 1.0, 166.0, 600.0))",Grand Village,No,No,3.8,166,Casual Dining,"North Indian, Rajasthani",600.0,Buffet,Banashankari
2.0,"List(0, 2761, List(0, 2, 5, 97, 2757, 2759, 2760), List(1.0, 1.0, 1.0, 1.0, 1.0, 286.0, 600.0))",Timepass Dinner,Yes,No,3.8,286,Casual Dining,North Indian,600.0,Buffet,Banashankari
3.0,"List(0, 2761, List(1, 2, 5, 1178, 2757, 2759, 2760), List(1.0, 1.0, 1.0, 1.0, 1.0, 8.0, 800.0))",Rosewood International Hotel - Bar & Restaurant,No,No,3.6,8,Casual Dining,"North Indian, South Indian, Andhra, Chinese",800.0,Buffet,Banashankari
19.0,"List(0, 2761, List(0, 3, 20, 178, 2757, 2759, 2760), List(1.0, 1.0, 1.0, 1.0, 1.0, 2556.0, 600.0))",Onesta,Yes,Yes,4.6,2556,"Casual Dining, Cafe","Pizza, Cafe, Italian",600.0,Cafes,Banashankari
4.0,"List(0, 2761, List(0, 2, 6, 286, 2757, 2759, 2760), List(1.0, 1.0, 1.0, 1.0, 1.0, 324.0, 700.0))",Penthouse Cafe,Yes,No,4.0,324,Cafe,"Cafe, Italian, Continental",700.0,Cafes,Banashankari
9.0,"List(0, 2761, List(0, 2, 6, 1216, 2757, 2759, 2760), List(1.0, 1.0, 1.0, 1.0, 1.0, 504.0, 550.0))",Smacznego,Yes,No,4.2,504,Cafe,"Cafe, Mexican, Italian, Momos, Beverages",550.0,Cafes,Banashankari


Finding the maximum and minimum limits of the average cost

In [76]:
max_average_cost=spark.sql("select max(average_cost) from `zomato` ")
min_average_cost=spark.sql("select min(average_cost) from `zomato` ")
max_average_cost.show()
min_average_cost.show()

+-----------------+
max(average_cost)|
+-----------------+
 6000.0|
+-----------------+

+-----------------+
min(average_cost)|
+-----------------+
 40.0|
+-----------------+

Finding the maximum rating from the total ratings

In [78]:
max_rate=spark.sql("select max(rate) from `zomato` ")
max_rate.show()

+---------+
max(rate)|
+---------+
 4.9|
+---------+

Finding restaurants that have a high rating and are cheaper where the average cost is less than 1500 Rupees

In [80]:
Cheapcosthighrating  = spark.sql("SELECT name,average_cost,rate,locality,rest_type,cuisines FROM `zomato`  WHERE rate>= 4.0 AND average_cost <1500 ORDER BY average_cost ASC ")
Cheapcosthighrating.show(50)

+--------------------+------------+----+--------------------+------------------+--------------------+
 name|average_cost|rate| locality| rest_type| cuisines|
+--------------------+------------+----+--------------------+------------------+--------------------+
 Shiv Shakthi| 100.0| 4.0| BTM| Quick Bites|Street Food, Nort...|
 Mumbai Kulfi| 100.0| 4.0| Bellandur| Dessert Parlor| Ice Cream|
 SLV Refreshment| 100.0| 4.1| Basavanagudi| Quick Bites| South Indian|
 By 2 Coffee| 100.0| 4.3| Basavanagudi| Quick Bites| South Indian|
 Poha On Wheels| 100.0| 4.1| Bellandur|Takeaway, Delivery| Street Food|
 Dakshin Cafe| 100.0| 4.1| Bannerghatta Road| Quick Bites| South Indian|
 Iyengar's Inn| 100.0| 4.1| Basavanagudi| Quick Bites| South Indian|
Sri Laxmi Venkate...| 100.0| 4.4| Banashankari| Quick Bites| South Indian|
 Taaza Thindi| 100.0| 4.7| Banashankari| Quick Bites| South Indian|
 Chai Kraft| 100.0| 4.1| Basavanagudi| Beverage Shop| Beverages, Tea|
 Puliyogare Point| 100.0| 4.2| Basavanagudi| Quick Bites| South Indian|
 South Kitchen| 100.0| 4.3| Basavanagudi| Quick Bites| South Indian|
Karnataka Bhel House| 100.0| 4.1| Basavanagudi| Quick Bites| Street Food|
Mahalaxmi Tiffin ...| 100.0| 4.1| Basavanagudi| Quick Bites| South Indian|
Sri Laxmi Venkate...| 100.0| 4.4| Basavanagudi| Quick Bites| South Indian|
Davangere Benna D...| 100.0| 4.1|Koramangala 7th B...| Quick Bites| South Indian|
 South Kitchen| 100.0| 4.3| Banashankari| Quick Bites| South Indian|
 Puliyogare Point| 100.0| 4.2| Banashankari| Quick Bites| South Indian|
Karnataka Bhel House| 100.0| 4.1| Banashankari| Quick Bites| Street Food|
 South Kitchen| 100.0| 4.3| Banashankari| Quick Bites| South Indian|
 Puliyogare Point| 100.0| 4.2| Banashankari| Quick Bites| South Indian|
Mahalaxmi Tiffin ...| 100.0| 4.1| Banashankari| Quick Bites| South Indian|
 SL Vadapav| 100.0| 4.0| Basavanagudi| Quick Bites| Maharashtrian|
Davangere Benna D...| 100.0| 4.1| Basavanagudi| Quick Bites| South Indian|
Karnataka Bhel House| 100.0| 4.1| Basavanagudi| Quick Bites| Street Food|
 By 2 Coffee| 100.0| 4.3| Rajajinagar| Quick Bites| South Indian|
Brahmin's Coffee Bar| 100.0| 4.8| Basavanagudi| Quick Bites| South Indian|
Karnataka Bhel House| 100.0| 4.1| Rajajinagar| Quick Bites| Street Food|
 By 2 Coffee| 100.0| 4.1| Malleshwaram| Quick Bites| South Indian|
 Amrith Ice Cream| 100.0| 4.1| Rajajinagar| Dessert Parlor| Ice Cream|
Brahmin Tiffins &...| 100.0| 4.2| Basavanagudi| Quick Bites| South Indian|
 Holige Mane| 100.0| 4.2| Rajajinagar| Quick Bites| South Indian|
 Amrith Ice Cream| 100.0| 4.1| Malleshwaram| Dessert Parlor| Ice Cream|
 Holige Mane| 100.0| 4.1| Rajajinagar| Quick Bites| South Indian|
 South Kitchen| 100.0| 4.3| Basavanagudi| Quick Bites| South Indian|
 By 2 Coffee| 100.0| 4.0| Residency Road| Quick Bites| South Indian|
 Holige Mane| 100.0| 4.3| Malleshwaram| Quick Bites| South Indian|
 Poha On Wheels| 100.0| 4.1| Sarjapur Road|Takeaway, Delivery| Street Food|
 Chai Kraft| 100.0| 4.1| Banashankari| Beverage Shop| Beverages, Tea|
 Mumbai Kulfi| 100.0| 4.0| Sarjapur Road| Dessert Parlor| Ice Cream|
 By 2 Coffee| 100.0| 4.0| Lavelle Road| Quick Bites| South Indian|
 Holige Mane| 100.0| 4.1| Malleshwaram| Quick Bites| South Indian|
 SLV Refreshment| 100.0| 4.1| Banashankari| Quick Bites| South Indian|
Sri Sairam's Chat...| 100.0| 4.2| Malleshwaram| Quick Bites| Street Food|
 SL Vadapav| 100.0| 4.0|Koramangala 7th B...| Quick Bites| Maharashtrian|
 Hari Super Sandwich| 100.0| 4.0|Koramangala 7th B...| Quick Bites| Sandwich, Fast Food|
 Puliyogare Point| 100.0| 4.2| Basavanagudi| Quick Bites| South Indian|
 Lassi Shop| 100.0| 4.0| Old Airport Road| Beverage Shop|Beverages, Juices...|
 Chai Kraft| 100.0| 4.1|Koramangala 7th B...| Beverage Shop| Beverages, Tea|
Brahmins' Thatte ...| 100.0| 4.1| Malleshwaram| Quick Bites| South Indian|
+--------------------+------------+----+--------------------+------------------+--------------------+
only showing top 50 rows

Finding the average votes obtained

In [82]:
mean_vote=spark.sql("select avg(votes) from `zomato` ")
mean_vote.show()

+-----------------+
 avg(votes)|
+-----------------+
296.7634506472492|
+-----------------+

Finding the most reliable restaurants that have rating above `4.0`, votes above the `average votes` and cost below `1500 Rupees` which is 1/4th of the maximum expense for a meal of two people

In [84]:
reliablerestaurant =  spark.sql("SELECT name,average_cost,votes,rate,locality,rest_type,cuisines FROM `zomato`  WHERE rate>= 4.0 AND average_cost <1500 AND votes >= 297 ORDER BY average_cost ASC , rate DESC ")
reliablerestaurant.show()

+--------------------+------------+-----+----+------------+-----------+------------+
 name|average_cost|votes|rate| locality| rest_type| cuisines|
+--------------------+------------+-----+----+------------+-----------+------------+
Brahmin's Coffee Bar| 100.0| 2679| 4.8|Basavanagudi|Quick Bites|South Indian|
 Taaza Thindi| 100.0| 651| 4.7|Banashankari|Quick Bites|South Indian|
Sri Laxmi Venkate...| 100.0| 343| 4.4|Banashankari|Quick Bites|South Indian|
Sri Laxmi Venkate...| 100.0| 343| 4.4|Basavanagudi|Quick Bites|South Indian|
Sri Laxmi Venkate...| 100.0| 344| 4.4| Jayanagar|Quick Bites|South Indian|
 By 2 Coffee| 100.0| 310| 4.3| Rajajinagar|Quick Bites|South Indian|
 By 2 Coffee| 100.0| 316| 4.3|Basavanagudi|Quick Bites|South Indian|
 By 2 Coffee| 100.0| 319| 4.3| Jayanagar|Quick Bites|South Indian|
Brahmin Tiffins &...| 100.0| 424| 4.2| JP Nagar|Quick Bites|South Indian|
 Puliyogare Point| 100.0| 449| 4.2| Jayanagar|Quick Bites|South Indian|
 Puliyogare Point| 100.0| 444| 4.2|Banashankari|Quick Bites|South Indian|
Brahmin Tiffins &...| 100.0| 421| 4.2|Basavanagudi|Quick Bites|South Indian|
 Puliyogare Point| 100.0| 444| 4.2|Basavanagudi|Quick Bites|South Indian|
 Puliyogare Point| 100.0| 444| 4.2|Basavanagudi|Quick Bites|South Indian|
Sri Sairam's Chat...| 100.0| 471| 4.2|Malleshwaram|Quick Bites| Street Food|
Brahmin Tiffins &...| 100.0| 424| 4.2| Jayanagar|Quick Bites|South Indian|
 Puliyogare Point| 100.0| 444| 4.2|Banashankari|Quick Bites|South Indian|
Karnataka Bhel House| 100.0| 449| 4.1|Basavanagudi|Quick Bites| Street Food|
Karnataka Bhel House| 100.0| 453| 4.1| Rajajinagar|Quick Bites| Street Food|
Karnataka Bhel House| 100.0| 449| 4.1|Banashankari|Quick Bites| Street Food|
+--------------------+------------+-----+----+------------+-----------+------------+
only showing top 20 rows

Finding the most expensived restaurants that have rating above `4.2` and cost above `3000 Rupees` which is half of the maximum expense for a meal of two people

In [86]:
expensive_restaurants = spark.sql("SELECT name,average_cost,votes,rate,locality,rest_type,cuisines FROM `zomato`  WHERE rate>= 4.2 AND average_cost>3000  ORDER BY  rate DESC ")
expensive_restaurants.show()

+--------------------+------------+-----+----+--------------+----------------+--------------------+
 name|average_cost|votes|rate| locality| rest_type| cuisines|
+--------------------+------------+-----+----+--------------+----------------+--------------------+
Karavalli - The G...| 3500.0| 675| 4.5| Church Street| Fine Dining|Mangalorean, Konk...|
Karavalli - The G...| 3500.0| 682| 4.5|Residency Road| Fine Dining|Mangalorean, Konk...|
Karavalli - The G...| 3500.0| 681| 4.5| MG Road| Fine Dining|Mangalorean, Konk...|
Alba - JW Marriot...| 4000.0| 583| 4.5| Brigade Road| Fine Dining| Italian|
Karavalli - The G...| 3500.0| 674| 4.5| Brigade Road| Fine Dining|Mangalorean, Konk...|
Karavalli - The G...| 3500.0| 681| 4.5| Lavelle Road| Fine Dining|Mangalorean, Konk...|
The Lantern Resta...| 3500.0| 424| 4.4| Church Street|Fine Dining, Bar| Chinese|
The Lantern Resta...| 3500.0| 424| 4.4| Brigade Road|Fine Dining, Bar| Chinese|
The Lantern Resta...| 3500.0| 424| 4.4| Church Street|Fine Dining, Bar| Chinese|
Blue Ginger - The...| 3200.0| 514| 4.4| Church Street| Fine Dining| Vietnamese|
Blue Ginger - The...| 3200.0| 516| 4.4| Lavelle Road| Fine Dining| Vietnamese|
Masala Klub - The...| 4000.0| 309| 4.4| Lavelle Road| Fine Dining|North Indian, Sou...|
Alba - JW Marriot...| 4000.0| 561| 4.4| Church Street| Fine Dining| Italian|
Alba - JW Marriot...| 4000.0| 571| 4.4| MG Road| Fine Dining| Italian|
Alba - JW Marriot...| 4000.0| 578| 4.4|Residency Road| Fine Dining| Italian|
The Lantern Resta...| 3500.0| 424| 4.4| Brigade Road|Fine Dining, Bar| Chinese|
Masala Klub - The...| 4000.0| 309| 4.4| Church Street| Fine Dining|North Indian, Sou...|
Edo Restaurant & ...| 4000.0| 237| 4.3| Lavelle Road| Fine Dining| Japanese|
The Lantern Resta...| 3500.0| 430| 4.3| Lavelle Road|Fine Dining, Bar| Chinese|
b CafÃƒÂƒÃ‚ÂƒÃƒÂ‚...| 3400.0| 429| 4.3| Lavelle Road|Fine Dining, Bar|North Indian, Ita...|
+--------------------+------------+-----+----+--------------+----------------+--------------------+
only showing top 20 rows